-What about electrons? (I don't see any - 2 phys.me)

-1e11 and mDM?

-Compute T(E,z_in, z_dep) by doing a delta function injection

# Full Transfer Functions

In [2]:
%load_ext autoreload
import sys
sys.path.append("..")

In [43]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from astropy.io import fits

import numpy as np
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_spectrum import nonrel_spec
from darkhistory.electrons.ics.ics_spectrum import rel_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

from darkhistory.electrons import positronium as pos

from darkhistory.low_energy.lowE_deposition import compute_fs

# np.set_printoptions(threshold=np.nan)

In [4]:
soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
xe_std  = interp1d(soln[0,:], soln[2,:])
T_m_std = interp1d(soln[0,:], soln[1,:])

## Photon Transfer Functions

In [5]:
user = 'hongwan'

highengphot_tflist_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_photspec_60eV_complete_coarse.raw", "rb"))
lowengphot_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec_60eV_complete_coarse.raw", "rb"))
lowengelec_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec_60eV_complete_coarse.raw", "rb"))
CMB_engloss_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/CMB_engloss_60eV_complete_coarse.raw", "rb"))

In [6]:
photeng = highengphot_tflist_arr[0].eng
eleceng = lowengelec_tflist_arr[0].eng

#Split photeng into high and low energy. 
photeng_high = photeng[photeng > 60]
photeng_low  = photeng[photeng <= 60]

# Split eleceng into high and low energy. 
eleceng_high = eleceng[eleceng > 3000]
eleceng_low  = eleceng[eleceng <= 3000]


for highengphot_tflist in highengphot_tflist_arr:
    
#     highengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)

    for tf in highengphot_tflist:
        # tf.rebin(photeng_high)
#         tf.rebin(photeng)
        
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size, 0), (0, 0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    highengphot_tflist._eng = photeng
    highengphot_tflist._in_eng = photeng
    highengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in highengphot_tflist._tflist])
    )
    
# lowengphot_tflist.in_eng set to photeng_high
for lowengphot_tflist in lowengphot_tflist_arr:
    
#     lowengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengphot_tflist:
#         tf.rebin(photeng)
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        # Photons in the low energy bins should be immediately deposited.
        tf._grid_vals[0:photeng_low.size, 0:photeng_low.size] = np.identity(photeng_low.size)
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    lowengphot_tflist._eng = photeng
    lowengphot_tflist._in_eng = photeng
    lowengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengphot_tflist._tflist])
    )
    
# lowengelec_tflist.in_eng set to photeng_high 
for lowengelec_tflist in lowengelec_tflist_arr:
    
#     lowengelec_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengelec_tflist:
#         tf.rebin(eleceng)
        # Pad with zeros so that it becomes photeng x eleceng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = eleceng
        tf._rs = tf.rs[0]*np.ones_like(photeng)

    lowengelec_tflist._eng = eleceng
    lowengelec_tflist._in_eng = photeng
    lowengelec_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengelec_tflist._tflist])
    )
    
    for engloss in CMB_engloss_arr:
        engloss = np.pad(engloss, ((0,0),(photeng_low.size, 0)), 'constant')


In [7]:
xes = 0.5 + 0.5*np.tanh([-5., -4.1, -3.2, -2.3, -1.4, -0.5, 0.4, 1.3, 2.2, 3.1, 4])

highengphot_tf_interp = tflist.TransferFuncInterp(xes, highengphot_tflist_arr)
lowengphot_tf_interp = tflist.TransferFuncInterp(xes, lowengphot_tflist_arr)
lowengelec_tf_interp = tflist.TransferFuncInterp(xes, lowengelec_tflist_arr)


In [26]:
phys.CMB_eng_density(phys.TCMB(1000))/1e11

2.60570190763316

## Inverse Compton Scattering

Set up all of the inverse Compton scattering transfer functions (needs to be hidden away at some point). 

In [8]:
Emax = 1e20
Emin = 1e-8
nEe = 500
nEp  = 500

dlnEp = np.log(Emax/Emin)/nEp
lowengEp_rel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)        

dlnEe = np.log(Emax/Emin)/nEe
lowengEe_rel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

Emax = 1e10
Emin = 1e-8
nEe = 500
nEp  = 500

dlnEp = np.log(Emax/Emin)/nEp
lowengEp_nonrel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)  

dlnEe = np.log(Emax/Emin)/nEe
lowengEe_nonrel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

print('********* Thomson regime scattered photon spectrum *********')
ics_thomson_ref_tf = nonrel_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(1000))
print('********* Relativistic regime scattered photon spectrum *********')
ics_rel_ref_tf = rel_spec(lowengEe_rel, lowengEp_rel, phys.TCMB(1000), inf_upp_bound=True)
print('********* Thomson regime energy loss spectrum *********')
engloss_ref_tf = engloss_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(1000), nonrel=True)

********* Thomson regime scattered photon spectrum *********
Initializing...
Computing spectra by an expansion in beta...
----> Computation by expansion in beta complete!
Computing spectra by analytic series...
*** Computing series 1/12...
*** Computing series 2/12...
*** Computing series 3/12...
*** Computing series 4/12...
*** Computing series 5/12...
*** Computing series 6/12...
*** Computing series 7/12...
*** Computing series 8/12...
*** Computing series 9/12...
*** Computing series 10/12...
*** Computing series 11/12...
*** Computing series 12/12...
----> Computation by analytic series complete!
Spectrum computed!
********* Relativistic regime scattered photon spectrum *********
Initializing...
Computing series 1/4...
Computing series 2/4...
Computing series 3/4...
Computing series 4/4...
Relativistic Computation Complete!
********* Thomson regime energy loss spectrum *********
Computing nonrelativistic energy loss spectrum...
Computing energy loss spectrum by beta expansion...
C

In [44]:
(ics_sec_phot_tf, ics_sec_elec_tf) = get_ics_cooling_tf(
        ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf,
        eleceng, photeng, 1000
    )

-------- Injection Energy:  256.3934702575207
-------- No. of Scatters (Analytic):  0.008191350379652062
-------- No. of Scatters (Computed):  0.008196841258664062
-------- Energy lost (Analytic):  6.954921801530184e-06
-------- Energy lost (Computed from photons):  6.97546554469112e-06
-------- Energy lost (Computed from electrons):  2.036028381493614
-------- Deposited Energy:  2.036008905744185
---- continuum engloss:  0.633988088794575
***************************************************
injected energy:  256.3934702575207
low energy e:  7.5463032714132225
scattered phot:  341.1340230217462
continuum_engloss:  339.8603233948345
diff:  1.2736996269117071
deposited:  133153.47853963106
***************************************************
-------- Injection Energy:  271.8930080339778
-------- No. of Scatters (Analytic):  0.008191350379652062
-------- No. of Scatters (Computed):  0.008196892183782327
-------- Energy lost (Analytic):  7.375473637600904e-06
-------- Energy lost (Computed f

***************************************************
injected energy:  2682.564162788971
low energy e:  1.199292602618122e-49
scattered phot:  1230.964446818953
continuum_engloss:  1211.166547441565
diff:  19.79789937738815
deposited:  4160713.801258064
***************************************************
-------- Injection Energy:  2844.730947057018
-------- No. of Scatters (Analytic):  0.008191350379652062
-------- No. of Scatters (Computed):  0.008196714147607212
-------- Energy lost (Analytic):  7.73614725128038e-05
-------- Energy lost (Computed from photons):  7.755995836209004e-05
-------- Energy lost (Computed from electrons):  20.903652332776623
-------- Deposited Energy:  20.903562476126524
---- continuum engloss:  0.6339979204044794
***************************************************
injected energy:  2844.730947057018
low energy e:  1.2415332006587235e-50
scattered phot:  1271.707000235258
continuum_engloss:  1250.2080108052116
diff:  21.498989430046322
deposited:  4524851.02

---- continuum engloss:  0.6340279492173303
***************************************************
injected energy:  16549.06896426488
low energy e:  2.3687064072712636e-74
scattered phot:  3739.181514221669
continuum_engloss:  3434.497762910739
diff:  304.68375131093035
deposited:  51729282.821459256
***************************************************
-------- Injection Energy:  17549.49584455794
-------- No. of Scatters (Analytic):  0.008191350379652062
-------- No. of Scatters (Computed):  0.008196305506295583
-------- Energy lost (Analytic):  0.00048410022792517705
-------- Energy lost (Computed from photons):  0.00048557208820691305
-------- Energy lost (Computed from electrons):  107.64948617098173
-------- Deposited Energy:  107.64898889139081
---- continuum engloss:  0.6340295294924478
***************************************************
injected energy:  17549.49584455794
low energy e:  5.959933053346953e-75
scattered phot:  3892.5574016990076
continuum_engloss:  3558.107690676623

diff:  7602.603017957053
deposited:  435503191.47545254
***************************************************
-------- Injection Energy:  121750.35886465822
-------- No. of Scatters (Analytic):  0.008191350379652062
-------- No. of Scatters (Computed):  0.008196136881655342
-------- Energy lost (Analytic):  0.003695107212222921
-------- Energy lost (Computed from photons):  0.003703910895290929
-------- Energy lost (Computed from electrons):  421.1262377769534
-------- Deposited Energy:  421.1225216907884
---- continuum engloss:  0.6340425738086735
***************************************************
injected energy:  121750.35886465822
low energy e:  2.529555945650407e-88
scattered phot:  21058.49586372042
continuum_engloss:  12639.628387205319
diff:  8418.867476515103
deposited:  456156249.0578181
***************************************************
-------- Injection Energy:  129110.43041651015
-------- No. of Scatters (Analytic):  0.008191350379652062
-------- No. of Scatters (Computed

-------- No. of Scatters (Computed):  0.008195997167321398
-------- Energy lost (Analytic):  0.04183735199464262
-------- Energy lost (Computed from photons):  0.042012046836546774
-------- Energy lost (Computed from electrons):  612.1452866254167
-------- Deposited Energy:  612.103262100342
---- continuum engloss:  0.6340533821135182
***************************************************
injected energy:  844648.1890401265
low energy e:  3.123516173121812e-92
scattered phot:  297090.73408432584
continuum_engloss:  45782.89064884923
diff:  251307.84343547662
deposited:  640021601.9556997
***************************************************
-------- Injection Energy:  895708.9921905147
-------- No. of Scatters (Analytic):  0.008191350379652062
-------- No. of Scatters (Computed):  0.008196022321913257
-------- Energy lost (Analytic):  0.04558012505273194
-------- Energy lost (Computed from photons):  0.04570810602824435
-------- Energy lost (Computed from electrons):  605.4655102710758
----

...

***************************************************
injected energy:  680234814.24638
low energy e:  1.101990702728898e-92
scattered phot:  678732948.5563565
continuum_engloss:  104313.33434307236
diff:  678628635.2220135
deposited:  1606212.010550619
***************************************************
-------- Injection Energy:  721356474.5979489
-------- No. of Scatters (Analytic):  0.008191350379652062
-------- No. of Scatters (Computed):  0.00816432046714575
-------- Energy lost (Analytic):  13827.417633898625
-------- Energy lost (Computed from photons):  13798.04273260929
-------- Energy lost (Computed from electrons):  13798.042732616886
-------- Deposited Energy:  0.005196707319555571
---- continuum engloss:  0.0
***************************************************
injected energy:  721356474.5979489
low energy e:  1.1019907027288924e-92
scattered phot:  719854608.9079235
continuum_engloss:  104313.33434307184
diff:  719750295.5735804
deposited:  1606209.9993506444
*************

-------- No. of Scatters (Analytic):  0.008191350379652062
-------- No. of Scatters (Computed):  0.007923916533289898
-------- Energy lost (Analytic):  840583.0059125819
-------- Energy lost (Computed from photons):  759950.1316482429
-------- Energy lost (Computed from electrons):  759950.1316482276
-------- Deposited Energy:  0.005196684505790472
---- continuum engloss:  0.0
***************************************************
injected energy:  5627787588.545746
low energy e:  1.1019907027288772e-92
scattered phot:  5626285722.855709
continuum_engloss:  104313.3343430703
diff:  5626181409.521366
deposited:  1606183.6363835603
***************************************************
-------- Injection Energy:  5967999475.529718
-------- No. of Scatters (Analytic):  0.008191350379652062
-------- No. of Scatters (Computed):  0.007908613210929723
-------- Energy lost (Analytic):  945275.2095619654
-------- Energy lost (Computed from photons):  844579.0823195329
-------- Energy lost (Computed f

-------- Deposited Energy:  0.005196660757064819
---- continuum engloss:  0.0
***************************************************
injected energy:  39043010955.325554
low energy e:  1.1019907027288726e-92
scattered phot:  39041509089.635506
continuum_engloss:  104313.33434306984
diff:  39041404776.30116
deposited:  1606181.1738113463
***************************************************
-------- Injection Energy:  41403245100.92151
-------- No. of Scatters (Analytic):  0.008191350379652062
-------- No. of Scatters (Computed):  0.006729015483327017
-------- Energy lost (Analytic):  45488941.79065025
-------- Energy lost (Computed from photons):  24752348.476965018
-------- Energy lost (Computed from electrons):  24752348.4769651
-------- Deposited Energy:  0.00519677996635437
---- continuum engloss:  0.0
***************************************************
injected energy:  41403245100.92151
low energy e:  1.1019907027288721e-92
scattered phot:  41401743235.231445
continuum_engloss:  10431

***************************************************
injected energy:  408494732299.3396
low energy e:  1.1014935157317553e-92
scattered phot:  408513387601.29285
continuum_engloss:  104266.27112072788
diff:  408513283335.0217
deposited:  -9584887.060012616
***************************************************
-------- Injection Energy:  433189119127.5538
-------- No. of Scatters (Analytic):  0.008191350379652062
-------- No. of Scatters (Computed):  0.003583687791963488
-------- Energy lost (Analytic):  4979462365.544918
-------- Energy lost (Computed from photons):  481999522.12808025
-------- Energy lost (Computed from electrons):  481959459.3489138
-------- Deposited Energy:  -40062.77396976948
---- continuum engloss:  0.0
***************************************************
injected energy:  433189119127.5538
low energy e:  1.1011659752354557e-92
scattered phot:  433227432370.3835
continuum_engloss:  104235.26646595466
diff:  433227328135.117
deposited:  -19448181.180455104
**********

-------- Energy lost (Computed from electrons):  1923893334.3657138
-------- Deposited Energy:  0.005197048187255859
---- continuum engloss:  0.0
***************************************************
injected energy:  2520054353269.385
low energy e:  1.094659757690372e-92
scattered phot:  2520754345874.3535
continuum_engloss:  103619.39444052974
diff:  2520754242254.959
deposited:  -699888978.5651071
***************************************************
-------- Injection Energy:  2672397069361.388
-------- No. of Scatters (Analytic):  0.008191350379652062
-------- No. of Scatters (Computed):  0.0014770841048706595
-------- Energy lost (Analytic):  189508411188.10632
-------- Energy lost (Computed from photons):  1979304507.9763641
-------- Energy lost (Computed from electrons):  1978012612.8414874
-------- Deposited Energy:  -1291895.1296799183
---- continuum engloss:  0.0
***************************************************
injected energy:  2672397069361.388
low energy e:  1.068698173443

## Evolution

In [10]:
# mDM = 605662.1322589782*2
# mDM = 10**9.104*2
# mDM = 1.27164e9*2
mDM = 10**8*2
# mDM = 10**6.5*2

phot_spec_init = spectools.rebin_N_arr(np.array([2]), 
                                      np.array([mDM/2]), 
                                      photeng)
phot_spec_init.switch_spec_type()

elec_spec_init = spectools.rebin_N_arr(np.array([2]), 
    np.array([mDM/2 - phys.me]),
    photeng)

elec_spec_init.switch_spec_type()

In [11]:
print(phot_spec_init.toteng(bound_type = 'eng', bound_arr = np.array([1,1e13]))/mDM)

[1.]


In [12]:
%autoreload
phot_spec_init.rs = 3000
elec_spec_init.rs = 3000
end_rs   = 100
#mDM = 1e8
sigmav = 1e-26
tau = 1e25

def rate_func_swave_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)/(2*mDM)

def rate_func_swave_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)

def rate_func_decay_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('decay', rs, tau=tau)/mDM

def rate_func_decay_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('decay', rs, tau=tau)


def rate_func_swave_N_2(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)/(2*mDM)

def rate_func_swave_eng_2(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)


def evolve(
    in_spec_elec, in_spec_phot, 
    rate_func_N, rate_func_eng, xe_init, T_m_init,
    highengphot_tf_interp, lowengphot_tf_interp, 
    lowengelec_tf_interp, 
    end_rs, coarsen_factor=1, std_soln=False
):
    """
    Main function that computes the temperature and ionization history. 
    
    Parameters
    ----------
    in_spec_elec : Spectrum
        Spectrum per annihilation/decay into electrons. 
    in_spec_phot : Spectrum
        Spectrum per annihilation/decay into photons.
    rate_func_N : function
        Function describing the rate of annihilation/decay, dN/(dV dt)
    rate_func_eng : function
        Function describing the rate of annihilation/decay, dE/(dV dt)
    xe_init : float
        xe at the initial redshift. 
    T_m_init : float
        Matter temperature at the initial redshift. 
    highengphot_tf_interp : TransferFuncInterp
        Interpolation table for high energy photons. 
    lowengphot_tf_interp : TransferFuncInterp
        Interpolation table for low energy photons. 
    lowengelec_tf_interp : TransferFuncInterp
        Interpolation table for low energy electrons. 
    end_rs : float
        Final redshift to evolve to. 
    coarsen_factor : int
        Coarsening to apply to the transfer function matrix. 
    std_soln : bool
        If true, uses the standard TLA solution for f(z). 
    """
    
    # Initialize the next spectrum as None.
    next_highengphot_spec = None
    next_lowengphot_spec  = None
    next_lowengelec_spec  = None
    
    if (
        highengphot_tf_interp.dlnz    != lowengphot_tf_interp.dlnz
        or highengphot_tf_interp.dlnz != lowengelec_tf_interp.dlnz
        or lowengphot_tf_interp.dlnz  != lowengelec_tf_interp.dlnz
    ):
        raise TypeError('TransferFuncInterp objects must all have the same dlnz.')
    
    if in_spec_elec.rs != in_spec_phot.rs:
        raise TypeError('Input spectra must have the same rs.')
    
    # redshift/timestep related quantities. 
    dlnz = highengphot_tf_interp.dlnz
    prev_rs = None
    rs = in_spec_phot.rs
    dt = dlnz/phys.hubble(rs)
    
    # Function that changes the normalization from per annihilation to per baryon. 
    def norm_fac(rs): 
        return rate_func_N(rs) * dt / (phys.nB * rs**3)
    
    # ICS for in_spec_elec goes here.
    
    (ics_sec_phot_tf, ics_sec_elec_tf) = get_ics_cooling_tf(
        ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf,
        eleceng, photeng, rs
    )
    
    ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec) * norm_fac(rs)
    ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec) * norm_fac(rs)
    positronium_phot_spec = pos.weighted_photon_spec(photeng)*in_spec_elec.totN()/2*norm_fac(rs)
    if positronium_phot_spec.spec_type != 'N':
        positronium_phot_spec.switch_spec_type()
        
    print('input electron energy: ', (phys.me*2 + in_spec_elec.toteng())*norm_fac(rs))
    print('ICS secondary photon spectrum energy: ', 
          ics_phot_spec.toteng(bound_type='eng', bound_arr=np.array([phys.rydberg, np.inf]))
         )
    print('ICS low energy electron spectrum energy: ', ics_lowengelec_spec.toteng() + phys.me*norm_fac(rs))
    print('Positronium spectrum energy: ', positronium_phot_spec.toteng())
    
    init_inj_spec = in_spec_phot * norm_fac(rs) + ics_phot_spec + positronium_phot_spec
    
    # Initialize the Spectra object that will contain all the 
    # output spectra during the evolution.
    out_highengphot_specs = Spectra([init_inj_spec], spec_type=init_inj_spec.spec_type)
    out_lowengphot_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    out_lowengelec_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    
    # Initialize the xe and T array that will store the solutions.
    xe_arr  = np.array([xe_init])
    T_m_arr = np.array([T_m_init])
    
    # Load the standard TLA solution if necessary. 
#     if std_soln:
    if True:
        soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
        xe_std  = interp1d(soln[0,:], soln[2,:])
        T_m_std = interp1d(soln[0,:], soln[1,:])

    # Define these methods for speed.
    append_highengphot_spec = out_highengphot_specs.append
    append_lowengphot_spec  = out_lowengphot_specs.append
    append_lowengelec_spec  = out_lowengelec_specs.append
    
    # Loop while we are still at a redshift above end_rs.
    while rs > end_rs:
        
        # If prev_rs exists, calculate xe and T_m. 
        if prev_rs is not None:
#             print('here at least?')
            
            # f_continuum, f_lyman, f_ionH, f_ionHe, f_heat
            # f_raw takes in dE/(dV dt)
            if std_soln:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )
#                 print("\n SHOULD BE ONE: ", (next_lowengelec_spec.toteng() + next_lowengphot_spec.toteng())/(
#                 np.sum(f_raw) * rate_func_eng(rs) * dt / phys.nB / rs**3), '\n')   
            else:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
                
            std_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )
            
            backreaction_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
            
            f_raw = np.float64(f_raw)
            
            init_cond = np.array([T_m_arr[-1], xe_arr[-1], 0, 0])

            new_vals = tla.get_history(
                init_cond, f_raw[2], f_raw[1], f_raw[4], 
                rate_func_eng, np.array([prev_rs, rs]),
                reion_switch = False
            )

            T_m_arr = np.append(T_m_arr, new_vals[-1,0])
            xe_arr  = np.append(xe_arr,  new_vals[-1,1])
            
        print('T_m at '+str(rs)+': '+ str(T_m_arr[-1]))
        print('x_e at '+str(rs)+': '+ str(xe_arr[-1]))
        print('Standard x_e at '+str(rs)+': '+str(xe_std(rs)))
        print('Standard T_m at '+str(rs)+': '+str(T_m_std(rs)))
        if prev_rs is not None:
            print('Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', backreaction_f)
            print('Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', std_f)
            
        if std_soln:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_std(rs))
        else:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_arr[-1])
        
#         if coarsen_factor > 1:
#             prop_tf = np.zeros_like(highengphot_tf._grid_vals)
#             for i in np.arange(coarsen_factor):
#                 prop_tf += matrix_power(highengphot_tf._grid_vals, i)
#             lowengphot_tf._grid_vals = np.matmul(prop_tf, lowengphot_tf._grid_vals)
#             lowengelec_tf._grid_vals = np.matmul(prof_tf, lowengelec_tf._grid_vals)
#             highengphot_tf._grid_vals = matrix_power(
#                 highengphot_tf._grid_vals, coarsen_factor
#             )
            
        next_highengphot_spec = highengphot_tf.sum_specs(out_highengphot_specs[-1])
        next_lowengphot_spec  = lowengphot_tf.sum_specs(out_highengphot_specs[-1])
        next_lowengelec_spec  = lowengelec_tf.sum_specs(out_highengphot_specs[-1]) + ics_lowengelec_spec
        
#         print('\n SHOULD ALSO BE ONE: ', 
#               out_highengphot_specs[-1].toteng(bound_type = 'eng', bound_arr = np.array([100,1e13]))/(
#         next_highengphot_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13])) + 
#             next_lowengphot_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13])) + 
#             next_lowengelec_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13]))), '\n')
        
#         print('ENERGIES:')
#         print(out_highengphot_specs[-1].toteng())
#         for i in np.arange(500):
#             print(i, highengphot_tf.in_eng[i], highengphot_tf[i].toteng())
    
#         print('\n SHOULD ALSO BE ONE (JUST HIGH ENERGY): ', 
#               out_highengphot_specs[-1].toteng()/(
#         next_highengphot_spec.toteng()), '\n')
        
#         output_eng = next_highengphot_spec.N*next_highengphot_spec.eng/norm_fac(rs)
#         output_N   = next_highengphot_spec.dNdE/norm_fac(rs)
#         utils.compare_arr([next_highengphot_spec.eng, output_eng])
#         print(output_eng[385:391]/1e9)
#         print(output_N[385:391])
#         print(next_highengphot_spec.eng[385:391])
        
#         utils.compare_arr([out_highengphot_specs[-1].N, next_highengphot_spec.N])
        
        # Re-define existing variables.
        prev_rs = rs
        rs = np.exp(np.log(rs) - 0.002)
        
        dt = dlnz/phys.hubble(rs)
        next_highengphot_spec.rs = rs
        next_lowengphot_spec.rs  = rs
        next_lowengelec_spec.rs  = rs
        
        # Add the next injection spectrum to next_highengphot_spec
    
        (ics_sec_phot_tf, ics_sec_elec_tf) = get_ics_cooling_tf(
            ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf,
            eleceng, photeng, rs
        )

        ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec) * norm_fac(rs)
        ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec) * norm_fac(rs)
        positronium_phot_spec = pos.weighted_photon_spec(photeng)*in_spec_elec.totN()/2 * norm_fac(rs)
        if positronium_phot_spec.spec_type != 'N':
            positronium_phot_spec.switch_spec_type()

        next_inj_spec = in_spec_phot * norm_fac(rs) + ics_phot_spec + positronium_phot_spec

        # This keeps the redshift. 
        next_highengphot_spec.N += next_inj_spec.N
        
        append_highengphot_spec(next_highengphot_spec)
        append_lowengphot_spec(next_lowengphot_spec)
        append_lowengelec_spec(next_lowengelec_spec)
        
        print('************************ END OF LOOP **************************')
        
        
    return (
        xe_arr, T_m_arr, 
        out_highengphot_specs, out_lowengphot_specs, out_lowengelec_specs
    )

In [13]:
%%prun
# a = evolve(
#         phot_spec_init*0, phot_spec_init, 
#         rate_func_decay_N, rate_func_decay_eng, .9999, phys.TCMB(phot_spec_init.rs),
#         highengphot_tf_interp, lowengphot_tf_interp, 
#         lowengelec_tf_interp, 
#         end_rs, coarsen_factor=1, std_soln=False
#     )

b_t = evolve(
        elec_spec_init, phot_spec_init*0, 
        rate_func_decay_N, rate_func_decay_eng, xe_std(phot_spec_init.rs), phys.TCMB(phot_spec_init.rs),
        #rate_func_decay_N, rate_func_decay_eng, 2e-4, phys.TCMB(phot_spec_init.rs),
        highengphot_tf_interp, lowengphot_tf_interp, 
        lowengelec_tf_interp, 
        end_rs, coarsen_factor=1, std_soln=True
    )

***************************************************
injected energy:  256.3934702575207
low energy e:  241.7774994142828
scattered phot:  10923.711828268653
continuum_engloss:  10886.987479210637
diff:  36.72434905801674
deposited:  -22.108378214774763
***************************************************
***************************************************
injected energy:  271.8930080339778
low energy e:  241.77749941427751
scattered phot:  21848.025770525666
continuum_engloss:  21773.651959831208
diff:  74.37381069445837
deposited:  -44.25830207477189
***************************************************
***************************************************
injected energy:  288.3295262688189
low energy e:  241.77749941427544
scattered phot:  32779.505446054965
continuum_engloss:  32666.54674939772
diff:  112.95869665724604
deposited:  -66.406669802703
***************************************************
***************************************************
injected energy:  305.7596674498054

***************************************************
injected energy:  3016.701062885404
low energy e:  241.7774994143306
scattered phot:  471617.9465645378
continuum_engloss:  467859.42967129184
diff:  3758.5168932459783
deposited:  -983.5933297747973
***************************************************
***************************************************
injected energy:  3199.0671427921043
low energy e:  241.77749941433765
scattered phot:  482682.8609219012
continuum_engloss:  478719.9123493922
diff:  3962.9485725089908
deposited:  -1005.658929131025
***************************************************
***************************************************
injected energy:  3392.457645207818
low energy e:  241.7774994143534
scattered phot:  493753.5558679356
continuum_engloss:  489575.1496068221
diff:  4178.406261113472
deposited:  -1027.726115319798
***************************************************
***************************************************
injected energy:  3597.5390202295966


diff:  52313.3509797029
deposited:  -2076.904563805688
***************************************************
***************************************************
injected energy:  53529.7413259219
low energy e:  241.7774994147574
scattered phot:  1047781.7714329736
continuum_engloss:  992392.7535946787
diff:  55389.01783829485
deposited:  -2101.0540117692076
***************************************************
***************************************************
injected energy:  56765.7295397133
low energy e:  241.7774994147948
scattered phot:  1061360.9004578886
continuum_engloss:  1002711.7466170149
diff:  58649.15384087374
deposited:  -2125.2018005595905
***************************************************
***************************************************
injected energy:  60197.34021422308
low energy e:  241.777499414831
scattered phot:  1075099.46039057
continuum_engloss:  1012994.5792531398
diff:  62104.88113743032
deposited:  -2149.31842262504
**************************************

***************************************************
injected energy:  1132742.6425017721
low energy e:  241.77749941490734
scattered phot:  2562694.771698007
continuum_engloss:  1427098.7546368553
diff:  1135596.0170611516
deposited:  -3095.152058430953
***************************************************
***************************************************
injected energy:  1201219.3761754245
low energy e:  241.77749941493974
scattered phot:  2636182.357762167
continuum_engloss:  1432098.453935324
diff:  1204083.9038268432
deposited:  -3106.305150428852
***************************************************
***************************************************
injected energy:  1273835.6759593948
low energy e:  241.7774994149161
scattered phot:  2713651.974279285
continuum_engloss:  1436940.7043587188
diff:  1276711.2699205661
deposited:  -3117.3714605484115
***************************************************
***************************************************
injected energy:  1350841.78754

deposited:  -2435.0038578920153
***************************************************
***************************************************
injected energy:  25418998.423860718
low energy e:  241.77749941497333
scattered phot:  26947139.4385219
continuum_engloss:  1525062.4275028822
diff:  25422077.011019018
deposited:  -2485.1682292607165
***************************************************
***************************************************
injected energy:  26955631.653697785
low energy e:  241.77749941497243
scattered phot:  28483772.668358006
continuum_engloss:  1525062.4275028766
diff:  26958710.240855128
deposited:  -2530.7596477417646
***************************************************
***************************************************
injected energy:  28585157.673552223
low energy e:  241.77749941497686
scattered phot:  30113298.688212413
continuum_engloss:  1525062.4275029043
diff:  28588236.26070951
deposited:  -2572.750314714204
************************************************

...

***************************************************
injected energy:  2240927210034.778
low energy e:  241.77749941503654
scattered phot:  2277502356941.0356
continuum_engloss:  1525079.3554775682
diff:  2277500831861.68
deposited:  -36573622029.92805
***************************************************
***************************************************
injected energy:  2376396088830.3022
low energy e:  241.77749941503654
scattered phot:  2421650749382.947
continuum_engloss:  1525079.355477568
diff:  2421649224303.5913
deposited:  -45253135674.242874
***************************************************
***************************************************
injected energy:  2520054353269.385
low energy e:  241.7774994150366
scattered phot:  2536641630867.12
continuum_engloss:  1525079.3554775678
diff:  2536640105787.7646
deposited:  -16585752717.654707
***************************************************
***************************************************
injected energy:  2672397069361.38

***************************************************
injected energy:  1491.5551945846528
low energy e:  241.77749941428266
scattered phot:  339557.9274648404
continuum_engloss:  337442.59234478185
diff:  2115.3351200585603
deposited:  -865.5574248881978
***************************************************
***************************************************
injected energy:  1581.7229202328017
low energy e:  241.77749941428584
scattered phot:  350546.243393222
continuum_engloss:  348314.06672936847
diff:  2232.176663853519
deposited:  -892.2312430349697
***************************************************
***************************************************
injected energy:  1677.341479197843
low energy e:  241.77749941427413
scattered phot:  361540.22695748997
continuum_engloss:  359185.754773864
diff:  2354.4721836259705
deposited:  -918.9082038424298
***************************************************
***************************************************
injected energy:  1778.74038609972

***************************************************
injected energy:  7276.0966711129295
low energy e:  241.77749941426003
scattered phot:  639102.4285939361
continuum_engloss:  630482.0503554508
diff:  8620.378238485311
deposited:  -1586.059066787216
***************************************************
***************************************************
injected energy:  7715.952394060849
low energy e:  241.77749941428291
scattered phot:  650377.9826219616
continuum_engloss:  641291.1773352114
diff:  9086.805286750197
deposited:  -1612.6303921027516
***************************************************
***************************************************
injected energy:  8182.398343301087
low energy e:  241.7774994142949
scattered phot:  661673.5235396787
continuum_engloss:  652093.7303560751
diff:  9579.793183603557
deposited:  -1639.1723397169897
***************************************************
***************************************************
injected energy:  8677.041955312132
l

***************************************************
injected energy:  39915.332374940335
low energy e:  241.77749941407725
scattered phot:  982394.1041382418
continuum_engloss:  940383.0446995872
diff:  42011.05943865469
deposited:  -2337.504563157964
***************************************************
***************************************************
injected energy:  42328.30023010768
low energy e:  241.77749941396786
scattered phot:  995289.5382439075
continuum_engloss:  950841.1130297978
diff:  44448.42521410971
deposited:  -2361.9024834355178
***************************************************
***************************************************
injected energy:  44887.1371917972
low energy e:  241.77749941394876
scattered phot:  1008310.9036794067
continuum_engloss:  961279.4402513799
diff:  47031.46342802688
deposited:  -2386.103735646405
***************************************************
***************************************************
injected energy:  47600.66136183002
l

KeyboardInterrupt: 

In [ ]:
plt.figure()
# plt.plot(a[3].rs, a[0])
plt.plot(b[3].rs, b[0])
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


plt.figure()
# plt.plot(a[3].rs, a[1]/phys.kB)
plt.plot(b[3].rs, b[1]/phys.kB)
plt.plot(b[3].rs, phys.TCMB(b[3].rs)/phys.kB, 'k:')
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


In [ ]:
test_tf = highengphot_tf_interp.get_tf(4, xe_std(3000))
highengphot_tf_interp.xe

In [ ]:
plt.figure()

ax = plt.gca()

print(test_tf.in_eng[375])

test_tf.plot(ax, ind=250, indtype='ind')
test_tf.plot(ax, ind=377, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
test_spectra = highengphot_tf_interp._grid_vals[-1,-1,:,:]

plt.figure()
ax = plt.gca()

plt.plot(highengphot_tf_interp.eng, test_spectra[370, :])

ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
tf_before_interp = highengphot_tflist_arr[-1][-1]
tf_before_interp_2 = highengphot_tflist_arr[-1][0]

# print(tf_before_interp.in_eng[376])

plt.figure()
ax = plt.gca()
# tf_before_interp.plot(ax, ind=200, indtype='ind')
tf_before_interp.plot(ax, ind=370, indtype='ind')
# tf_before_interp_2.plot(ax, ind=376, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
lowengphot_spectra = b[3]
print(lowengphot_spectra.in_eng)

lowengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengphot_spectra[0].toteng())

plt.figure()
ax = plt.gca()

lowengphot_spectra.switch_spec_type()
lowengphot_spectra.plot(ax, ind=0, indtype='ind')
lowengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4, 1e3, 1e-10, 1e6])

In [ ]:
lowengelec_spectra = b[4]

lowengelec_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengelec_spectra[0].toteng())
utils.compare_arr([lowengelec_spectra[0].eng, lowengelec_spectra[0].dNdE])

plt.figure()
ax = plt.gca()

lowengelec_spectra.switch_spec_type()
lowengelec_spectra.plot(ax, ind=0, indtype='ind')
lowengelec_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1, 3e3, 1e-4, 1e3])

In [ ]:
highengphot_spectra = b[2]

highengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(2*10**9.104)
print(highengphot_spectra[0].toteng())
print(highengphot_spectra[1].toteng())

plt.figure()
ax = plt.gca()

highengphot_spectra.switch_spec_type()
highengphot_spectra.plot(ax, ind=1, indtype='ind')
highengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4,5e12, 1e-40, 1e10])